<a href="https://colab.research.google.com/github/fernandodeeke/epidemias/blob/main/sir_rec_disc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Configurações do matplotlib
plt.rc('text', usetex=False)
plt.rc('font', family='serif')
plt.rc('mathtext', fontset='cm')

# Função modelo SIR discreto
def sir_disc(beta, gamma, nu, mu, alpha, Lambda, N0, I0, R0, T):
    passos = int(T) + 1
    S = np.zeros(passos)
    I = np.zeros(passos)
    R = np.zeros(passos)
    N = np.zeros(passos)
    t = np.arange(0, T+1)

    S[0] = max(0, N0 - I0 - R0)
    I[0] = I0
    R[0] = R0
    N[0] = S[0] + I[0] + R[0]

    max_threshold = 1e8

    for k in range(passos - 1):
        N[k] = S[k] + I[k] + R[k]
        if N[k] > max_threshold:
            S[k+1:] = np.nan
            I[k+1:] = np.nan
            R[k+1:] = np.nan
            N[k+1:] = np.nan
            break
        S[k+1] = S[k] + (Lambda + nu * N[k] - beta * S[k] * I[k] / N[k] - mu * S[k])
        I[k+1] = I[k] + (beta * S[k] * I[k] / N[k] - gamma * I[k] - (mu + alpha) * I[k])
        R[k+1] = R[k] + (gamma * I[k] - mu * R[k])
        N[k+1] = S[k+1] + I[k+1] + R[k+1]
    return t, S, I, R, N

# Função de plotagem com valores de equilíbrio no gráfico
def plot_sir(beta, gamma, nu, mu, alpha, Lambda, N0, I0, R0, T):
    t, S, I, R, N = sir_disc(beta, gamma, nu, mu, alpha, Lambda, N0, I0, R0, T)

    # Pega os últimos valores válidos
    I_eq = I[~np.isnan(I)][-1]
    N_eq = N[~np.isnan(N)][-1]

    plt.figure(figsize=(14, 10), dpi=200)

    # I(t)
    plt.subplot(2, 2, 1)
    plt.plot(t, I, color='blue')
    plt.xlabel('$t$ (dias)')
    plt.ylabel('$I(t)$')
    plt.title('Série temporal de $I(t)$')
    plt.grid(True)
    plt.text(0.7 * T, 0.9 * max(I), f'$I^* = {I_eq:.2f}$', fontsize=12, bbox=dict(facecolor='white', alpha=0.7))

    # N(t)
    plt.subplot(2, 2, 2)
    plt.plot(t, N, color='blue')
    plt.xlabel('$t$ (dias)')
    plt.ylabel('$N(t)$')
    plt.title('Série temporal de $N(t)$')
    plt.grid(True)
    plt.text(0.7 * T, 0.9 * max(N), f'$N^* = {N_eq:.2f}$', fontsize=12, bbox=dict(facecolor='white', alpha=0.7))

    # Diagrama de fase
    plt.subplot(2, 2, 3)
    plt.plot(N, I, color='blue')
    plt.plot(N_eq, I_eq, 'ro', label='Equilíbrio')
    plt.xlabel('$N(t)$')
    plt.ylabel('$I(t)$')
    plt.title('Diagrama de fase: $I(t)$ vs. $N(t)$')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Sliders dos parâmetros
beta_slider = widgets.FloatSlider(value=1.065, min=0.01, max=1.5, step=0.001, description='beta')
gamma_slider = widgets.FloatSlider(value=0.113, min=0.001, max=1.0, step=0.001, description='gamma')
nu_slider = widgets.FloatSlider(value=0.0274, min=0.0, max=0.1, step=0.0001, description='nu')
mu_slider = widgets.FloatSlider(value=0.0262, min=0.0001, max=0.1, step=0.0001, description='mu')
alpha_slider = widgets.FloatSlider(value=0.0223, min=0.001, max=0.1, step=0.0001, description='alpha')
Lambda_slider = widgets.FloatSlider(value=13, min=0, max=100, step=1, description='Lambda')
N0_slider = widgets.IntSlider(value=10000, min=100, max=10000, step=10, description='N0')
I0_slider = widgets.IntSlider(value=10, min=0, max=500, step=1, description='I0')
R0_slider = widgets.IntSlider(value=0, min=0, max=500, step=1, description='R0')
T_slider = widgets.IntSlider(value=260, min=50, max=2000, step=10, description='T')

# Interface gráfica
ui = widgets.VBox([
    beta_slider, gamma_slider, nu_slider, mu_slider, alpha_slider,
    Lambda_slider, N0_slider, I0_slider, R0_slider, T_slider
])

out = widgets.interactive_output(plot_sir, {
    'beta': beta_slider,
    'gamma': gamma_slider,
    'nu': nu_slider,
    'mu': mu_slider,
    'alpha': alpha_slider,
    'Lambda': Lambda_slider,
    'N0': N0_slider,
    'I0': I0_slider,
    'R0': R0_slider,
    'T': T_slider
})

display(ui, out)


Output()